Imports

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import pandas as pd
from datasets import load_dataset
import transformers


In [2]:
# Let's login to huggingface ig to get results
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/d/.cache/huggingface/token
Login successful


Initialize Model

In [2]:
# Parse the training data into captions, frames, and scene graph from TVQA dataset
# Encode the individual parts
# Combine them into one embedding space
# Train a vanilla Flan-T5 based transformer
# Evaluate 

Collect training data

In [7]:
train_data_questions = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_qa_release/tvqa_train.jsonl"
train_data_frames = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/uncompressed_frames/frames_hq" # All frames
train_data_audio = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/uncompressed_audio" # All audio
subtitles = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_preprocessed_subtitles.jsonl" # All subtitles

In [8]:
test_data_questions = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_qa_release/tvqa_test_public.jsonl"
val_data_questions = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_qa_release/tvqa_val.jsonl"


In [5]:
# questions_df_train = pd.read_json(train_data_questions, lines=True)
# questions_df_test = pd.read_json(test_data_questions, lines=True)
subtitles_df = pd.read_json(subtitles, lines=True)
subtitles_df = subtitles_df.set_index('vid_name')


In [5]:
model_name = 'google/flan-t5-xxl'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
# Reading in datasets
# Should we read into dataframe or dataset?
# questions_df = pd.read_json(train_data_questions, lines=True)
# subtitles_df = pd.read_json(train_data_subtitles, lines=True)

# 
dataset = load_dataset('json', data_files = {"train": train_data_questions, "val": val_data_questions})

Found cached dataset json (/home/d/.cache/huggingface/datasets/json/default-448cf64c9f4da10c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
sample = dataset['train'][0]

In [34]:
import torch
from transformers import T5EncoderModel, T5Tokenizer, logging


# # Load the CLIP tokenizer
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# # Load the CLIP model
# model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
device = 'cuda:0' if torch.cuda.is_available() else "cpu"
model_id = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5EncoderModel.from_pretrained("google/flan-t5-large", torch_dtype=torch.float16).to(device)

def get_last_hidden_layer(sentence):
    # Tokenize the sentence and convert it to a PyTorch tensor
    tokens =tokenizer(sentence, return_tensors="pt", padding=False, truncation=False).to(device)

    # Generate the last hidden layer of the CLIP encoder
    lhs = model(**tokens).last_hidden_state

    return lhs

For stability purposes, it is recommended to have accelerate installed when using this model in torch.float16, please install it with `pip install accelerate`
Some weights of the model checkpoint at google/flan-t5-large were not used when initializing T5EncoderModel: ['decoder.block.22.layer.1.EncDecAttention.v.weight', 'decoder.block.6.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.0.layer_norm.weight', 'decoder.block.9.layer.2.DenseReluDense.wo.weight', 'decoder.block.12.layer.0.layer_norm.weight', 'decoder.block.22.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.8.layer.2.layer_norm.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.7.layer.2.layer_norm.weight', 'decoder.block.21.layer.0.SelfAttention.k.weight', 'decoder.block.2.layer.1.layer_norm.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'decoder.block.16.layer.0.SelfAttention.q.weight', 'decoder.block.16.layer.1.EncDecAttent

In [12]:
get_last_hidden_layer("3 inches is above average").shape

torch.Size([1, 6, 512])

In [13]:
def get_subtitle_from_video(vid_name: str, ts: str):
  if type(ts) != str:
    print(type(ts))
    print(ts)
  start_time, end_time = ts.split('-')
  subtitle = ''
  sub = subtitles_df.loc[vid_name]
  for text in sub['sub']:
    if text['start'] >= float(start_time) and text['end'] <= float(end_time):
      subtitle += text['text'] + ' '
  return subtitle.strip()
def get_all_subtitles(vid_name: str):
  sub = ' '.join([_dict["text"] for _dict in subtitles_df.loc[vid_name]["sub"]])
  return sub.strip()


In [14]:
get_all_subtitles("house_s02e05_seg02_clip_11")

'Chase : That\'s all this is? Yeah.  House : Because his white blood cell count was down, he was vulnerable.  House : Because it\'s really down, it might kill him.  Chase : That\'s all this is.  House : Is he still pooping his pants? It just started again. Again?  Chase : It stopped when he was getting better. When he was feeling better. He was never getting better.  Foreman : So, all we have to answer is, what causes a 22-year-old kid  Foreman : to become immunocompromised  Foreman : with GI involvement and shocks? Just stop and start for no apparent reason.  Cameron : And we need to find an answer  Cameron : before this infection kills him.  House : That would be the ideal. You. Intravenous broad spectrum antibiotics. You get a cervical, thoracic, and lumbar T2-weighted fast spin-echo MRI. And you, track down all those Richie Rich\'s who went to Jamaica,  House : see if any of them have got the shocks, the trots or the hots. The hots, it\'s the fever. You guys gonna join us for dinne

In [27]:
def output_text_encoding(question_sample):
  '''
  param: 
  dictionary with ['a0',
 'a1',
 'a2',
 'a3',
 'a4',
 'answer_idx',
 'q',
 'qid',
 'show_name',
 'ts',
 'vid_name']
    
    

  returns: list of last hidden state of encoding, (prompt, subtitles, answer) for each answer
  '''
  all_encodings = []
  for i in range(5):
    subtitles = get_all_subtitles(question_sample["vid_name"])
    ans_candidate = question_sample[f"a{i}"]
    prompt = f"Context: {subtitles}. Question: {question_sample['q']} Is it '{ans_candidate}'?"
    all_encodings.append(get_last_hidden_layer(prompt))
  return all_encodings

    

In [35]:
for i in range(10):
  sample = dataset['train'][i]
  print(output_text_encoding(sample)[0].shape)


torch.Size([1, 382, 1024])
torch.Size([1, 351, 1024])


Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 258, 1024])
torch.Size([1, 345, 1024])
torch.Size([1, 619, 1024])
torch.Size([1, 300, 1024])
torch.Size([1, 208, 1024])
torch.Size([1, 518, 1024])
torch.Size([1, 505, 1024])
torch.Size([1, 227, 1024])


In [9]:
list(dataset['train'].features.keys())

['a0',
 'a1',
 'a2',
 'a3',
 'a4',
 'answer_idx',
 'q',
 'qid',
 'show_name',
 'ts',
 'vid_name']

In [10]:
from datasets import concatenate_datasets
input = f"You are given the following dialogue: . 'Given the question , which of the following options is the most likely answer?\n"
prompt_size = len(tokenizer(input, truncation=True))

# The maximum question length
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["val"]]).map(lambda x: tokenizer(x["q"], truncation=True), batched=True, remove_columns=list(dataset['train'].features.keys()))
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length + prompt_size}")

# # The maximum answer length
# # Sequences longer than this will be truncated, sequences shorter will be padded.
# tokenized_inputs = concatenate_datasets([dataset["train"], dataset["val"]]).map(lambda x: tokenizer(x[f"a{x["answer_idx"]- 1}"], truncation=True), batched=True, remove_columns=list(dataset['train'].features.keys()))
# max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
# print(f"Max source length: {max_source_length}")


Loading cached processed dataset at /home/d/.cache/huggingface/datasets/json/default-448cf64c9f4da10c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-77b7e2af7910ad4a.arrow


Max source length: 69


In [11]:

from random import randrange
sample = dataset['train'][randrange(len(dataset["train"]))]
print(sample['a0'])

He was lying on the floor.


**Subtitles**

In [13]:
# Test cell
question =  sample
video = question['vid_name']
test_sentence = get_subtitle_from_video(video, question['ts'])
print(test_sentence)

House : That was the whole idea.  Wilson : You wanted to kill yourself?  House : I wanted to nearly kill myself.  House : Is he better? No.


In [14]:
dataset['train'].shape

(122039, 11)

Convert subtitles to CLIP embeddings

In [15]:
# Train model
def preprocess_function(sample, padding="max_length"):
    # Convert to relevant information
    a0, a1, a2, a3, a4, answer_idx, q, ts, vid_name = sample['a0'], sample['a1'], sample['a2'], sample['a3'], sample['a4'], sample['answer_idx'], sample['q'], sample['ts'], sample['vid_name']
    # Get the subtitle given the video and time stamp
    subtitle = get_subtitle_from_video(vid_name, ts)
    # Create an answer list
    answer_list = [a0, a1, a2, a3, a4]
    # Convert the answer id to relevant answer
    answer = answer_list[answer_idx - 1]
    # add prefix to the input for t5
    input = f"You are given the following dialogue: '{subtitle}'. 'Given the question '{q}', which of the following options is the most likely answer?\n{answer_list}"

    # tokenize inputs
    model_inputs = tokenizer(input)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(answer)
    # labels = tokenizer(text_target=answer,  padding=padding, truncation=True)

    # # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # # padding in the loss.
    # if padding == "max_length":
    #     labels["input_ids"] = [
    #         [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    #     ]
    model_inputs['labels'] = labels['input_ids']

    return model_inputs


# preprocess_function(dataset['val'][1])
# Should be batching, but I'm not because some of the modifications I make are on an iterative level; should definitely change for scaling purposes.
tokenized_dataset = dataset.map(preprocess_function, batched = False, remove_columns=
                                ['qid', 'show_name', 'a0', 'a1', 'a2', 'a3', 'a4', 'answer_idx', 'q', 'ts', 'vid_name'], desc='Processing Dataset')

Loading cached processed dataset at /home/d/.cache/huggingface/datasets/json/default-448cf64c9f4da10c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-1caa1af1fc903c62.arrow


Processing Dataset:   0%|          | 0/15253 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors


In [16]:
tokenized_dataset['train'].features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [17]:
_input = torch.tensor(tokenized_dataset['train'][0]['input_ids']).unsqueeze(0)
_label = torch.tensor(tokenized_dataset['train'][0]['labels']).unsqueeze(0)
# print(_input)
# print(_label)
# print(_input)
# outputs = model(input_ids=_input)


# input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
# labels = tokenizer("Das Haus ist wunderbar.", return_tensors="pt").input_ids
# print(input_ids)
# print(labels)
# loss = model(input_ids=input_ids, labels=labels).loss

loss = model(input_ids=_input, labels=_label).loss
loss
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

tensor(2.9318, grad_fn=<NllLossBackward0>)

In [18]:
import evaluate
# import nltk
import numpy as np
# from nltk.tokenize import sent_tokenize
# nltk.download("punkt")

# Metric
metric = evaluate.load("exact_match")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # result = {k: round(v * 100, 4) for k, v in result.items()}
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    # result["gen_len"] = np.mean(prediction_lens)
    return result

In [19]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [20]:
transformers.__version__

'4.27.1'

In [21]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = "premo/vpt_tvqa"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    # Wrong version?
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token()
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    compute_metrics=compute_metrics,
)

In [22]:
# Start training
trainer.train()

/home/d/utils/miniconda3/envs/tvqa_finetune/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/d/utils/miniconda3/envs/tvqa_finetune/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to t

Epoch,Training Loss,Validation Loss


TypeError: _compute() got an unexpected keyword argument 'use_stemmer'

In [ ]:
trainer.evaluate()

In [ ]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

In [ ]:
# Evaluate model
def qa_to_prompt(qa):
  '''
  TODO: update prompt to be VPT-compatible. 
  TODO: add </s> token to end of prompt... during training.
  '''

  subtitle = get_subtitle_from_video(qa['vid_name'], float(qa['ts'].split('-')[0]), float(qa['ts'].split('-')[-1]))

  # image_embed = get_clip_embed_from_vid_name(qa['vid_name'], float(qa['ts'].split('-')[0]), float(qa['ts'].split('-')[-1]))

  return [
      f"Context: {subtitle}. Question: {qa['q']} Is it '{ans_candidate}'?"
      for ans_candidate in [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']]
  ]

def run_prompts_get_best_answer(prompts):
  all_yes_scores = []

  with torch.no_grad():
    for prompt in prompts:
      # print(prompt)
      inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Batch size 1
      outputs = model.generate(**inputs, return_dict_in_generate=True, output_scores=True, early_stopping=True, max_new_tokens=2)
      # print(outputs)
      all_yes_scores.append(outputs.scores[0][0][4273].cpu())
      # print('yes_score =', outputs.scores[0][0][4273])
      # print('no_score =', outputs.scores[0][0][150])
      # print(self.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=False))

def accuracy(actual, predicted):
  correct = 0
  total = len(actual)
  for i in range(total):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / total

In [ ]:
actual = []
predicted = []
for i in range(5):
  predicted_ans_idx = eval.run_prompts_get_best_answer(eval.qa_to_prompt(eval.train_qa_json[i]))
  actual.append(int(eval.train_qa_json[i]['answer_idx']))
  predicted.append(predicted_ans_idx)
  print("Predicted answer:", predicted_ans_idx)
  print("Actual answer:   ", eval.train_qa_json[i]['answer_idx'])

print("Accuracy:", eval.accuracy(actual, predicted))

from clip_encoder import ClipEncoder


# **Main Evaluation Code**

In [23]:
from clip_encoder import ClipEncoder


In [22]:
import json
import os
import pathlib
import sys
from typing import Any, Dict, List

import more_itertools
import numpy as np
import pandas as pd
import torch
from PIL import Image

os.environ['TRANSFORMERS_CACHE'] = '/mnt/teton/utils/cache/huggingface'
os.environ['HF_DATASETS_CACHE'] = '/mnt/teton/utils/cache/datasets'

sys.path.append("../../data_preprocessing/parallel_processing")
from clip_encoder import ClipEncoder
from text_encoder import FlanT5Encoder


class TVQA_eval():

  def __init__(self):

    train_filepath = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_qa_release/tvqa_train.jsonl"
    with open(train_filepath, 'r') as f:
      self.train_qa_json = [json.loads(line) for line in f]
    # len(train_qa_json)  # 122,039

    subtitles_filepath = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_preprocessed_subtitles.jsonl"
    with open(subtitles_filepath, 'r') as f:
      # self.subtitles = [json.loads(line) for line in f]
      self.subtitles = pd.read_json(subtitles_filepath, lines=True)
      self.subtitles = self.subtitles.set_index('vid_name')
    # len(subtitles)  # 21,793

    # instantiate expert models
    self.clip_encoder = ClipEncoder(debug=True)
    self.text_encoder = FlanT5Encoder()

    self.tvqa_train_to_path = {
        "House M.D.": 'house_frames',
        "The Big Bang Theory": 'bbt_frames',
        "Castle": 'castle_frames',
        "How I Met You Mother": 'met_frames',
        "Grey's Anatomy": 'grey_frames',
        "Friends": 'friends_frames',
    }

    self.vid_name_prefix_to_path = {
        "house": 'house_frames',
        "castle": 'castle_frames',
        "met": 'met_frames',
        "grey": 'grey_frames',
        "friends": 'friends_frames',
        "": 'bbt_frames',  # no prefix at all used for bbt, it's the "default"
    }

  # Deprecated
  def get_subtitle_from_clip(self, vid_name: str, ts: str):
    start_time, end_time = ts.split('-')
    subtitle = ''
    sub = self.subtitles.loc[vid_name]
    for text in sub['sub']:
      if text['start'] >= float(start_time) and text['end'] <= float(end_time):
        subtitle += text['text'] + ' '
    return subtitle.strip()

  def get_all_subtitles(self, vid_name: str):
    '''Returns a string with every subtitle from a given video'''
    sub = ' '.join([_dict["text"] for _dict in self.subtitles.loc[vid_name]["sub"]])
    return sub.strip()

  def qa_to_prompt(self, qa: Dict):
    '''
    TODO: update prompt to be VPT-compatible. 
    TODO: add </s> token to end of prompt... during training.
    '''
    # In this version, we only get the subtitles relevant to the time stamp
    # subtitle = self.get_subtitle_from_clip(qa['vid_name'], float(qa['ts'].split('-')[0]), float(qa['ts'].split('-')[-1]))
    # In this version, we get all subtitles from the video
    print(qa)
    print(qa['vid_name'])
    subtitle = self.get_all_subtitles(qa['vid_name'])

    # Should change this to all frames. Either way, this is irrelevant for this function
    # image_embed = self.get_clip_embed_from_vid_name(qa['vid_name'], float(qa['ts'].split('-')[0]), float(qa['ts'].split('-')[-1]))

    return [
        f"Context: {subtitle}. Question: {qa['q']} Is it '{ans_candidate}'?"
        for ans_candidate in [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']]
    ]

  def combine_modality_encodings(self, text_encoding, image_encoding):
    '''Untested btw'''
    num_text_embeddings, _ = text_encoding.shape
    num_image_embeddings, _ = image_encoding.shape

    assert num_image_embeddings + num_text_embeddings <= 1024, f"the given encoding has more than 1024 embeddings! Text is {num_text_embeddings} and image is {num_image_embeddings}"

    combined_tensor = torch.cat((text_encoding, image_encoding), dim=0)
    # Pad the tensor with -100 to make it a [1024, 1024] tensor

    # There shouldn't be any padding left
    # pad_size = (1024 - combined_tensor.shape[0], 1024)
    # padded_tensor = torch.nn.functional.pad(combined_tensor, pad_size, value=-100)

    # Reinsert 
    assert combined_tensor.shape[0] == 1024, f"the combined encoding does not have exactly 1024 embeddings! Dimension: {combined_tensor.shape}"

    return combined_tensor

  def create_context_vectors(self, question):
    '''Combine the two vectors to create the context vector'''
    try:
      all_context_vectors = []
      text_encodings = self.get_flant5_embed_from_vid_name(question)
      image_encoding = self.get_clip_embed_from_vid_name(question['vid_name'])  # this should be get_clip_embed_from_vid_name
      for text_encoding in text_encodings:
        all_context_vectors.append(self.combine_modality_encodings(text_encoding, image_encoding))
      return all_context_vectors
    except FileNotFoundError as e:
      print(e)
      print(f"WARNING: Could not find video {question['vid_name']}. Skipping...")

  def pad_or_truncate_tensor(self, tensor, truncate_shape):
      target_shape = [truncate_shape, 1024]
      tensor_shape = tensor.shape

      # If tensor shape is larger than the target shape, truncate the tensor
      if tensor_shape[0] > target_shape[0]:
          truncated_tensor = tensor[:target_shape[0], :]
          return truncated_tensor

      # If tensor shape is smaller than the target shape, pad the tensor
      elif tensor_shape[0] < target_shape[0]:
          padding_shape = (target_shape[0] - tensor_shape[0], target_shape[1])
          padded_tensor = torch.nn.functional.pad(tensor, (0, 0, 0, padding_shape[0]), value=-100)
          return padded_tensor

      # If tensor shape is already the target shape, return the tensor
      else:
          return tensor
      
  def vid_name_to_frames_path(self, vid_name):
    '''
    Example:
    vid_name: house_s02e05_seg02_clip_11
    
    show_name_path = house_frames
    clip_name_path = s02e05_seg02_clip_11
    
    returns: /mnt/teton/vpt/data/benchmark_datasets/TVQA/uncompressed_frames/frames_hq/house_frames/s02e05_seg02_clip_11
    '''

    show_name, clip_name_path = vid_name.split('_', 1)[0], vid_name.split('_', 1)[1]
    if show_name not in self.vid_name_prefix_to_path:
      show_name_path = 'bbt_frames'
      clip_name_path = vid_name  # no prefix at all used for bbt. vid_name is actually the 'clip_name_path'
    else:
      show_name_path = self.vid_name_prefix_to_path[show_name]

    frames_dir = os.path.join('/mnt/teton/vpt/data/benchmark_datasets/TVQA/uncompressed_frames/frames_hq', show_name_path, clip_name_path)
    if not os.path.exists(frames_dir):
      raise FileNotFoundError(f"frames_dir {frames_dir} does not exist")

    return frames_dir

  def get_clip_embed_from_vid_name(self, vid_name, max_encodings_to_make=220):
    '''
    ✅ Working
    
    PARAMS
    vid_name (from the subtitles jsonl file)
    max_encodings_to_make: int. It will only encode the FIRST max_encodings_to_make frames.
    
    RETURNS
    A list of clip embeddings for that video segment. There is a VARIABLE number of frames per "clip". 
    
    Notes: 
      * Castle avg frames/clip = 274.79
      * BBT avg frames/clip    = 186.38
    
    220 frames per clip is a good number to use.
    1024 (Flan-T5-XXL window size) - 220 = 804 text encodings to use. 
    '''
    # collect all frames from filepath
    segment_frames_paths = pathlib.Path(self.vid_name_to_frames_path(vid_name)).glob('*.jpg')
    frames_PIL_list = []
    for path in segment_frames_paths:
      if len(frames_PIL_list) >= max_encodings_to_make:
        break
      with Image.open(path) as img:
        frames_PIL_list.append(np.array(img))

    # split frames_PIL_list into batches of 100 (to avoid OOM)
    batch_list = list(more_itertools.batched(frames_PIL_list, 110))

    clip_embeddings = []
    for batch in batch_list:
      clip_embeddings.extend(self.clip_encoder.run_clip(batch, only_return_pooled_embeds=True))


    # Converting a list of tensors to a tensor
    # Get the shape of the first tensor in the list
    tensor_shape = clip_embeddings[0].shape
    # Create a tensor of zeros with the appropriate shape
    tensor = torch.zeros(len(clip_embeddings), *tensor_shape)
    # Fill the tensor with the values from the input list
    for i, t in enumerate(clip_embeddings):
        tensor[i, ...] = t
    fixed_tensor = self.pad_or_truncate_tensor(tensor, max_encodings_to_make)
    fixed_tensor.cpu()
    return fixed_tensor

  def get_flant5_embed_from_vid_name(self, question_sample, max_encodings_to_make=804):
    '''
    param: 
    question_sample is a dictionary with ['a0',  'a1',  'a2',  'a3', 'a4',    'answer_idx',   'q',   'qid',  'show_name', 'ts', 'vid_name'] as keys

    returns: list of last hidden state of encoding, (prompt, subtitles, answer) for each answer
    '''
    assert type(question_sample) == dict, f"question_sample must be a dictionary. It is a {type(question_sample)}."

    all_prompts = self.qa_to_prompt(question_sample)
    all_encodings = []
    for prompt in all_prompts:
      all_encodings.append(self.text_encoder.encode_tvqa(prompt, truncate_shape=max_encodings_to_make))
    return all_encodings

  def run_prompts_get_best_answer(self, prompts: List[str]):
    all_yes_scores = []

    with torch.no_grad():
      for prompt in prompts:
        # print(prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)  # Batch size 1
        outputs = self.model.generate(**inputs, return_dict_in_generate=True, output_scores=True, early_stopping=True, max_new_tokens=2)
        # print(outputs)
        all_yes_scores.append(outputs.scores[0][0][4273].cpu())
        # print('yes_score =', outputs.scores[0][0][4273])
        # print('no_score =', outputs.scores[0][0][150])
        # print(self.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=False))

    return np.array(all_yes_scores).argmax()

  def accuracy(self, actual, predicted):
    correct = 0
    total = len(actual)
    for i in range(total):
      if actual[i] == predicted[i]:
        correct += 1
    return correct / total





In [23]:
eval = TVQA_eval()

Using cuda:0...
Done setting up CLIP...
In FlanT5Encoder cuda:1


In [24]:
sample = dataset['train'][3]
sample

{'a0': 'Penny means that the man is turning Amy on. ',
 'a1': 'Penny means that her shoes are too tight. ',
 'a2': 'Penny means that she ate too much. ',
 'a3': 'Penny means that she is drunk. ',
 'a4': "Penny isn't sure what she means but it sounds good. ",
 'answer_idx': 0,
 'q': "What does Penny mean when she says she knows what's causing Amy's condition?",
 'qid': 3,
 'show_name': 'The Big Bang Theory',
 'ts': '43.42-46.3',
 'vid_name': 's04e10_seg01_clip_03'}

In [25]:
eval.create_context_vectors(sample)

{'a0': 'Penny means that the man is turning Amy on. ', 'a1': 'Penny means that her shoes are too tight. ', 'a2': 'Penny means that she ate too much. ', 'a3': 'Penny means that she is drunk. ', 'a4': "Penny isn't sure what she means but it sounds good. ", 'answer_idx': 0, 'q': "What does Penny mean when she says she knows what's causing Amy's condition?", 'qid': 3, 'show_name': 'The Big Bang Theory', 'ts': '43.42-46.3', 'vid_name': 's04e10_seg01_clip_03'}
s04e10_seg01_clip_03
⏰  Runtime of preprocessing: 0.60 seconds
RIGHT before running clip 📸
⏰ CLIP Runtime on 110 images: 0.01 seconds
⏰  Runtime of preprocessing: 0.30 seconds
RIGHT before running clip 📸
⏰ CLIP Runtime on 54 images: 0.01 seconds


[tensor[1024, 1024] n=1048576 x∈[-100.000, 6.544] μ=-67.114 σ=47.008,
 tensor[1024, 1024] n=1048576 x∈[-100.000, 6.544] μ=-67.163 σ=46.990,
 tensor[1024, 1024] n=1048576 x∈[-100.000, 6.544] μ=-67.163 σ=46.990,
 tensor[1024, 1024] n=1048576 x∈[-100.000, 6.544] μ=-67.261 σ=46.954,
 tensor[1024, 1024] n=1048576 x∈[-100.000, 6.544] μ=-66.919 σ=47.078]

In [ ]:
training_path = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_tvqa_whole/"


In [2]:
import deeplake as dl
import numpy as np
ds = dl.load("/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_tvqa_whole", read_only=True)
ds.summary()

print(ds.size_approx() / 1e9) # bytes to GB

DatasetCorruptError: The HEAD node of the branch main of this dataset is in a corrupted state and is likely not recoverable. Try using `reset=True` to reset HEAD changes and load the previous commit.

In [4]:
import os

In [3]:
frames_path = "/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq"


In [23]:
directory_items = os.listdir(frames_path)

all_dirs = [os.path.join(frames_path, show_path) for show_path in directory_items]

In [42]:
total_frames = 0
for i in all_dirs:
    length = len(os.listdir(i))
    total_frames += length
    print(i, length)
total_frames

/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/met_frames 1512
/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/castle_frames 4706
/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/friends_frames 5353
/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/grey_frames 1435
/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/bbt_frames 4199
/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/house_frames 4624


21829

In [74]:
4199 * 5

20995

In [63]:
all_met_clips = os.listdir(all_dirs[0])
all_met_clips.sort()
all_met_clips[-193]

'met_s06e15_seg02_clip_05'

In [73]:
x = os.path.join(all_dirs[0], "met_s06e15_seg02_clip_05")
os.path.exists(x)


True

In [72]:
os.path.exists("/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq/met_frames/s06e15_seg02_clip_05")

False

In [83]:
s1 = dataset['train'][0]
s2 = dataset['train'][1]
s3 = dataset['train'][2]

In [87]:
dataset['train'][3]

{'a0': 'Penny means that the man is turning Amy on. ',
 'a1': 'Penny means that her shoes are too tight. ',
 'a2': 'Penny means that she ate too much. ',
 'a3': 'Penny means that she is drunk. ',
 'a4': "Penny isn't sure what she means but it sounds good. ",
 'answer_idx': 0,
 'q': "What does Penny mean when she says she knows what's causing Amy's condition?",
 'qid': 3,
 'show_name': 'The Big Bang Theory',
 'ts': '43.42-46.3',
 'vid_name': 's04e10_seg01_clip_03'}

In [80]:

TVQA_FRAMES_DIR = '/mnt/teton/vpt/data/benchmark_datasets/TVQA/march_28_uncompressed_frames/frames_hq'

tvqa_train_to_path = {
    "House M.D.": 'house_frames',
    "The Big Bang Theory": 'bbt_frames',
    "Castle": 'castle_frames',
    "How I Met You Mother": 'met_frames',
    "Grey's Anatomy": 'grey_frames',
    "Friends": 'friends_frames',
}

vid_name_prefix_to_path = {
    "house": 'house_frames',
    "castle": 'castle_frames',
    "met": 'met_frames',
    "grey": 'grey_frames',
    "friends": 'friends_frames',
    "": 'bbt_frames',  # no prefix at all used for bbt, it's the "default"
}

def vid_name_to_frames_path(vid_name):
  '''
  Example:
  vid_name: house_s02e05_seg02_clip_11
  
  show_name_path = house_frames
  clip_name_path = s02e05_seg02_clip_11
  
  returns: /mnt/teton/vpt/data/benchmark_datasets/TVQA/uncompressed_frames/frames_hq/house_frames/s02e05_seg02_clip_11
  '''

  show_name, clip_name_path = vid_name.split('_', 1)[0], vid_name.split('_', 1)[1]
  if show_name not in vid_name_prefix_to_path:
    show_name_path = 'bbt_frames'
    # clip_name_path = vid_name  # no prefix at all used for bbt. vid_name is actually the 'clip_name_path'
  else:
    show_name_path = vid_name_prefix_to_path[show_name]

  frames_dir = os.path.join(TVQA_FRAMES_DIR, show_name_path, vid_name)
  if not os.path.exists(frames_dir):
    raise FileNotFoundError(f"frames_dir {frames_dir} does not exist")

  return frames_dir

In [82]:
os.path.exists(vid_name_to_frames_path(s1['vid_name']))

True